In [1]:
from database_setup import *
from model import *

from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.pipeline_options import PdfPipelineOptions, TableFormerMode
from docling.datamodel.base_models import InputFormat
from docling_core.types.doc import DoclingDocument


In [2]:
docling_docs = sorted(glob.glob("/workspace/src/docling_out/*"))[:50]

In [3]:
docling_docs[1]

'/workspace/src/docling_out/10.1002$hed.26257.json'

In [4]:
x = extract_data_from_json_docling(docling_docs[3])

In [5]:
db_vals = dotenv_values("/workspace/src/.env")
session = setup_engine_session(db_vals['USER'], db_vals['PASSWORD'], db_vals['ADDRESS'], db_vals['PORT'], db_vals['DB'], echo=False)

In [6]:
parallel_extract_data(docling_docs, model, session, num_workers=None, batch_size = 10)

Overall Progress:   2%|▏         | 1/50 [00:11<09:32, 11.69s/it]ERROR:root:Error committing session: (psycopg2.errors.UndefinedColumn) column "pdf" of relation "document" does not exist
LINE 1: INSERT INTO document (doi, title, publication_id, pdf, pdf_i...
                                                          ^

[SQL: INSERT INTO document (doi, title, publication_id, pdf, pdf_is_sane) VALUES (%(doi__0)s, %(title__0)s, %(publication_id__0)s, %(pdf__0)s, %(pdf_is_sane__0)s), (%(doi__1)s, %(title__1)s, %(publication_id__1)s, %(pdf__1)s, %(pdf_is_sane__1)s), (%(doi__2 ... 553 characters truncated ... pdf_is_sane__8)s), (%(doi__9)s, %(title__9)s, %(publication_id__9)s, %(pdf__9)s, %(pdf_is_sane__9)s)]
[parameters: {'title__0': '', 'publication_id__0': 101, 'pdf_is_sane__0': False, 'pdf__0': None, 'doi__0': '10.1002$ajh.21743', 'title__1': '', 'publication_id__1': 102, 'pdf_is_sane__1': False, 'pdf__1': None, 'doi__1': '10.1002$hed.26257', 'title__2': '', 'publication_id__2': 103, 'pdf_